In [2]:
# OUR TASK. Import request & beutiful soup
import requests
import bs4
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

def create_soup(url,tag_name):
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.text,"lxml")
    result = soup.select(tag_name)
    return result

def get_row_label(current_country, current_province):
    if current_province == '':
        result = current_country
    else:
        result = current_country+':-'+current_province
    return result

# SEARCH ITEMS hhhh
def find_exact(dictionary, search_string):
    result = dictionary.get(search_string)
    return result

def find_wildcard(dictionary,search_string):
    output_dictionary={}
    for key,val in dictionary.items():
        if search_string in key:
            output_dictionary[key] = val
    return output_dictionary


def transpose_for_dataframe(covstats_result):
    row_count = len(covstats_result[1])-1
    
    result={}
    headers = covstats_result[2]
    transposed_result = list(map(list, zip(*covstats_result[1])))  
    transposed_result_row_count = len(transposed_result) -1
    
    for n in range(transposed_result_row_count):   
        result.update({headers[n]:transposed_result[n]})
    return result

def transpose_for_dataframe2(covstats_result):
    row_count = len(covstats_result[2])

    result={}
    headers = covstats_result[3]
    transposed_result = list(map(list, zip(*covstats_result[2])))  
    transposed_result_len = len(transposed_result)
    print(row_count)
    print(transposed_result_len)
    #for n in range(row_count):   
     #   print(headers[n])

    

            
def get_covstats(url,tag):
    data = create_soup(url,tag) 
    #Create Resulting List
    country_lookup = []
    simple_list_output=[]
    #Create Header Row
    headers = data[0].getText()
    headers_list = headers.split('\n')
    headers_list_cleaned = headers_list[2:]

    # data[128:140] this will produce 11 rows
    for item in data[1:]:
        item_text = item.getText()
        item_list = item_text.split('\n')
        #item_list[3-5] is limiting the columns for testing purposes
        item_list_cleaned = item_list[2:]
    
        dictionary_row = dict(zip(headers_list_cleaned, item_list_cleaned))

        # put it into summary list
        current_province =  dictionary_row["Province/State"]
        current_country = dictionary_row["Country/Region"] 
        row_label = get_row_label(current_country, current_province)
        item_number = item.key
        #print(item_number)
        #1. build a SET of countries (FOR LOOKUP)
        country_lookup.append(row_label) 
        #3 Build just a list
        simple_list_output.append(item_list_cleaned)
        
    return [country_lookup,  simple_list_output,  headers_list_cleaned]

In [ ]:
#Create our Data
url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
tag = '.js-file-line'

covstats_result = get_covstats(url,tag)
input_data_for_dataframe = transpose_for_dataframe(covstats_result)

covid_dataframe = DataFrame(input_data_for_dataframe)
covid_dataframe

In [3]:
#Working with our dataframe
test_dataframe = covid_dataframe

,Province/State,Country/Region
0,,Afghanistan
1,,Albania
2,,Algeria
3,,Andorra
4,,Angola
...,...,...
266,,West Bank and Gaza
267,,Western Sahara
268,,Yemen
269,,Zambia


In [14]:
# GET UK
all_uk = test_dataframe.loc[test_dataframe['Country/Region'].isin(['United Kingdom'])]
uk_mainland = all_uk.loc[all_uk['Province/State'].isin([''])]

all_australia = test_dataframe.loc[test_dataframe['Country/Region'].isin(['Australia'])]

sweden = test_dataframe.loc[test_dataframe['Country/Region'].isin(['Sweden'])]


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,114,115,115,115,115,115,115,115,115,115
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,4502,4509,4514,4517,4527,4538,4542,4548,4552,4552
10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,0,...,46,46,46,46,46,46,46,47,48,49
11,Queensland,Australia,-27.4698,153.0251,0,0,0,0,0,0,...,1186,1187,1190,1190,1192,1193,1196,1197,1197,1197
12,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,0,...,547,551,551,553,554,555,556,557,558,560
13,Tasmania,Australia,-42.8821,147.3272,0,0,0,0,0,0,...,230,230,230,230,230,230,230,230,230,230
14,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,...,20345,20345,20345,20345,20345,20345,20345,20345,20345,20345
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,...,788,794,794,796,797,799,804,804,808,817


In [ ]:
sweden

In [ ]:
uk_mainland

In [ ]:
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([sweden, uk_mainland],axis=0)
vertical_stack